Ovo je prvi pokusaj KNN algoritma

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

In [2]:
df = pd.read_csv('../../preprocessing/apartments_preprocessed.csv')

In [3]:
df.head()

,bathrooms,bedrooms,price,square_meters,cityname,state,latitude,longitude,source,luxury
0,1.0,0.0,790,9.393,Washington,DC,38.9057,-76.9861,RentLingo,0
1,1.0,1.0,425,9.858,Evansville,IN,37.9680,-87.6621,RentLingo,0
2,1.0,0.0,1390,9.951,Arlington,VA,38.8910,-77.0816,RentLingo,0
3,1.0,0.0,925,10.788,Seattle,WA,47.6160,-122.3275,RentLingo,0
4,1.0,0.0,880,11.625,Arlington,VA,38.8738,-77.1055,RentLingo,0


In [5]:
from sklearn.model_selection import train_test_split

count     9922.000000
mean      1481.016831
std        948.711174
min        200.000000
25%        949.000000
50%       1270.000000
75%       1695.000000
max      25000.000000
Name: price, dtype: float64

KNN je algoritam koji koristi mere bliskosti da bi klasifikovao instance. Ono sto ja zelim je da procenim cenu apartmana, ali tako sto cu objekte kategorisati:
